In [7]:
#Task 1 (a)

In [1]:
import pulp

products = ['A', 'B', 'C']
months = [1, 2, 3, 4]


h = {'A': 1.2, 'B': 1.3, 'C': 1} 
c = {'A': 20, 'B': 30, 'C': 10} 
k = {'A': 1, 'B': 2, 'C': 0.5} 

H = {1: 500, 2: 450, 3: 400, 4: 350}

D = {
    'A': {1: 100, 2: 110, 3: 120, 4: 130},
    'B': {1: 80,  2: 90,  3: 100, 4: 110},
    'C': {1: 120, 2: 130, 3: 140, 4: 150}
}

model = pulp.LpProblem("Production_and_Inventory_Planning", pulp.LpMinimize)

# Decision variables
x = pulp.LpVariable.dicts("x", (products, months), lowBound=0, cat=pulp.LpInteger)

I = pulp.LpVariable.dicts("I", (products, months), lowBound=0, cat=pulp.LpInteger)

model += pulp.lpSum([
    c[p]*x[p][t] + k[p]*I[p][t]
    for p in products for t in months
]), "Total_Cost"


for p in products:
    for t in months:
        if t == 1:
            model += x[p][1] - D[p][1] == I[p][1], f"Inventory_Balance_{p}_{t}"
        else:
            model += I[p][t-1] + x[p][t] - D[p][t] == I[p][t], f"Inventory_Balance_{p}_{t}"

for t in months:
    model += sum(h[p]*x[p][t] for p in products) <= H[t], f"Hours_{t}"

model.solve(pulp.PULP_CBC_CMD(msg=0))

print("Status:", pulp.LpStatus[model.status])

print("\nOptimal Production Plan (x):")
for p in products:
    for t in months:
        print(f"x[{p},{t}] = {pulp.value(x[p][t])}", end=", ")
    print()

print("\nOptimal Inventory Plan (I):")
for p in products:
    for t in months:
        print(f"I[{p},{t}] = {pulp.value(I[p][t])}", end=", ")
    print()

print("\nTotal Minimized Cost = ", pulp.value(model.objective))


Status: Optimal

Optimal Production Plan (x):
x[A,1] = 100.0, x[A,2] = 110.0, x[A,3] = 120.0, x[A,4] = 130.0, 
x[B,1] = 80.0, x[B,2] = 90.0, x[B,3] = 100.0, x[B,4] = 110.0, 
x[C,1] = 162.0, x[C,2] = 201.0, x[C,3] = 126.0, x[C,4] = 51.0, 

Optimal Inventory Plan (I):
I[A,1] = 0.0, I[A,2] = 0.0, I[A,3] = 0.0, I[A,4] = 0.0, 
I[B,1] = 0.0, I[B,2] = 0.0, I[B,3] = 0.0, I[B,4] = 0.0, 
I[C,1] = 42.0, I[C,2] = 113.0, I[C,3] = 99.0, I[C,4] = 0.0, 

Total Minimized Cost =  26127.0


In [9]:
#Task 1 (b)

In [3]:
import pulp

products = ['A', 'B', 'C']
months = [1, 2, 3, 4]

h = {'A': 1.2, 'B': 1.3, 'C': 1}   # hours per unit

c = {
    'A': {1:20, 2:20, 3:20, 4:20},
    'B': {1:30, 2:30, 3:35, 4:30},
    'C': {1:10, 2:10, 3:10, 4:10}
}

k = {'A': 1, 'B': 2, 'C': 0.5} # holding cost per unit per month

H = {1: 500, 2: 450, 3: 400, 4: 350} # hours available per month

D = {
    'A': {1: 100, 2: 110, 3: 120, 4: 130},
    'B': {1: 80,  2: 90,  3: 100, 4: 110},
    'C': {1: 120, 2: 130, 3: 140, 4: 150}
}

model = pulp.LpProblem("Production_and_Inventory_Planning_Updated_Costs", pulp.LpMinimize)

# Decision variables
x = pulp.LpVariable.dicts("x", (products, months), lowBound=0, cat=pulp.LpInteger)
I = pulp.LpVariable.dicts("I", (products, months), lowBound=0, cat=pulp.LpInteger)

# Objective: Minimize total production + inventory holding cost
model += pulp.lpSum([c[p][t]*x[p][t] + k[p]*I[p][t]
                     for p in products for t in months]), "Total_Cost"

# Constraints

for p in products:
    for t in months:
        if t == 1:
            model += x[p][1] - D[p][1] == I[p][1], f"Inventory_Balance_{p}_{t}"
        else:
            model += I[p][t-1] + x[p][t] - D[p][t] == I[p][t], f"Inventory_Balance_{p}_{t}"

# Production hours constraints
for t in months:
    model += sum(h[p]*x[p][t] for p in products) <= H[t], f"Hours_{t}"

model.solve(pulp.PULP_CBC_CMD(msg=0))

print("Status:", pulp.LpStatus[model.status])

print("\nOptimal Production Plan (x):")
for p in products:
    for t in months:
        print(f"x[{p},{t}] = {pulp.value(x[p][t])}", end=", ")
    print()

print("\nOptimal Inventory Plan (I):")
for p in products:
    for t in months:
        print(f"I[{p},{t}] = {pulp.value(I[p][t])}", end=", ")
    print()

print("\nTotal Minimized Cost = ", pulp.value(model.objective))


Status: Optimal

Optimal Production Plan (x):
x[A,1] = 100.0, x[A,2] = 110.0, x[A,3] = 120.0, x[A,4] = 130.0, 
x[B,1] = 80.0, x[B,2] = 190.0, x[B,3] = 0.0, x[B,4] = 110.0, 
x[C,1] = 179.0, x[C,2] = 71.0, x[C,3] = 239.0, x[C,4] = 51.0, 

Optimal Inventory Plan (I):
I[A,1] = 0.0, I[A,2] = 0.0, I[A,3] = 0.0, I[A,4] = 0.0, 
I[B,1] = 0.0, I[B,2] = 100.0, I[B,3] = 0.0, I[B,4] = 0.0, 
I[C,1] = 59.0, I[C,2] = 0.0, I[C,3] = 99.0, I[C,4] = 0.0, 

Total Minimized Cost =  26279.0


In [11]:
#Task 1 (c)

In [5]:
import pulp

# Sets and indices
products = ['A', 'B', 'C']
months = [1, 2, 3, 4]

# Parameters
h = {'A': 1.2, 'B': 1.3, 'C': 1}   # hours per unit

# Month-dependent production costs (with updated cost for B in March)
c = {
    'A': {1:20, 2:20, 3:20, 4:20},
    'B': {1:30, 2:30, 3:30, 4:30},  # B cost in March is now 35
    'C': {1:10, 2:10, 3:10, 4:10}
}

k = {'A': 1, 'B': 2, 'C': 0.5} # holding cost per unit per month

H = {1: 500, 2: 450, 3: 400, 4: 350} # hours available per month

# Demand with updated C in April
D = {
    'A': {1: 100, 2: 110, 3: 120, 4: 130},
    'B': {1: 80,  2: 90,  3: 100, 4: 110},
    'C': {1: 120, 2: 130, 3: 140, 4: 170} 
}

# Create the model
model = pulp.LpProblem("Production_and_Inventory_Planning_Updated_Demand_and_Costs", pulp.LpMinimize)

# Decision variables
x = pulp.LpVariable.dicts("x", (products, months), lowBound=0, cat=pulp.LpInteger)
I = pulp.LpVariable.dicts("I", (products, months), lowBound=0, cat=pulp.LpInteger)

# Objective: Minimize total production + inventory holding cost
model += pulp.lpSum([c[p][t]*x[p][t] + k[p]*I[p][t]
                     for p in products for t in months]), "Total_Cost"

# Constraints
# Inventory balance constraints
for p in products:
    for t in months:
        if t == 1:
            # For the first month: x[p,1] - D[p,1] = I[p,1]
            model += x[p][1] - D[p][1] == I[p][1], f"Inventory_Balance_{p}_{t}"
        else:
            # For month t: I[p,t-1] + x[p][t] - D[p][t] = I[p][t]
            model += I[p][t-1] + x[p][t] - D[p][t] == I[p][t], f"Inventory_Balance_{p}_{t}"

# Production hours constraints
for t in months:
    model += sum(h[p]*x[p][t] for p in products) <= H[t], f"Hours_{t}"

# Solve the model
model.solve(pulp.PULP_CBC_CMD(msg=0))

# Print results
print("Status:", pulp.LpStatus[model.status])

print("\nOptimal Production Plan (x):")
for p in products:
    for t in months:
        print(f"x[{p},{t}] = {pulp.value(x[p][t])}", end=", ")
    print()

print("\nOptimal Inventory Plan (I):")
for p in products:
    for t in months:
        print(f"I[{p},{t}] = {pulp.value(I[p][t])}", end=", ")
    print()

print("\nTotal Minimized Cost = ", pulp.value(model.objective))


Status: Optimal

Optimal Production Plan (x):
x[A,1] = 100.0, x[A,2] = 110.0, x[A,3] = 120.0, x[A,4] = 130.0, 
x[B,1] = 80.0, x[B,2] = 90.0, x[B,3] = 100.0, x[B,4] = 110.0, 
x[C,1] = 182.0, x[C,2] = 201.0, x[C,3] = 126.0, x[C,4] = 51.0, 

Optimal Inventory Plan (I):
I[A,1] = 0.0, I[A,2] = 0.0, I[A,3] = 0.0, I[A,4] = 0.0, 
I[B,1] = 0.0, I[B,2] = 0.0, I[B,3] = 0.0, I[B,4] = 0.0, 
I[C,1] = 62.0, I[C,2] = 133.0, I[C,3] = 119.0, I[C,4] = 0.0, 

Total Minimized Cost =  26357.0


In [20]:
#Task 1 (d)

In [22]:
import pulp

# Sets and indices
products = ['A', 'B', 'C']
months = [1, 2, 3, 4]

# Parameters
h = {'A': 1.2, 'B': 1.3, 'C': 1}   # hours per unit

c = {
    'A': {1:20, 2:20, 3:20, 4:20},
    'B': {1:30, 2:30, 3:30, 4:30},  # B cost in March = 35 if we stick to the previous change
    'C': {1:10, 2:10, 3:10, 4:10}
}

k = {'A': 1, 'B': 2, 'C': 0.5} # holding cost per unit per month

H = {1: 500, 2: 450, 3: 400, 4: 350} # hours available per month

# Demand - same as previously changed scenario:
D = {
    'A': {1: 100, 2: 110, 3: 120, 4: 130},
    'B': {1: 80,  2: 90,  3: 100, 4: 110},
    'C': {1: 120, 2: 130, 3: 140, 4: 150}
}

# Create the model
model = pulp.LpProblem("Production_and_Inventory_with_Inventory_Cost_Cap", pulp.LpMinimize)

# Decision variables
x = pulp.LpVariable.dicts("x", (products, months), lowBound=0, cat=pulp.LpInteger)
I = pulp.LpVariable.dicts("I", (products, months), lowBound=0, cat=pulp.LpInteger)

# Objective: Minimize total (production + inventory) cost
model += pulp.lpSum([c[p][t]*x[p][t] + k[p]*I[p][t]
                     for p in products for t in months]), "Total_Cost"

# Constraints

# Inventory balance
for p in products:
    for t in months:
        if t == 1:
            # For the first month: x[p,1] - D[p,1] = I[p,1]
            model += x[p][1] - D[p][1] == I[p][1], f"Inventory_Balance_{p}_{t}"
        else:
            # For subsequent months: I[p,t-1] + x[p,t] - D[p,t] = I[p,t]
            model += I[p][t-1] + x[p][t] - D[p][t] == I[p][t], f"Inventory_Balance_{p}_{t}"

# Production hours constraint
for t in months:
    model += sum(h[p]*x[p][t] for p in products) <= H[t], f"Hours_{t}"

# Total inventory holding cost constraint
model += pulp.lpSum([k[p]*I[p][t] for p in products for t in months]) <= 500, "Inventory_Cost_Limit"

# Solve the model
model.solve(pulp.PULP_CBC_CMD(msg=0))

# Print results
print("Status:", pulp.LpStatus[model.status])

print("\nOptimal Production Plan (x):")
for p in products:
    for t in months:
        print(f"x[{p},{t}] = {pulp.value(x[p][t])}", end=", ")
    print()

print("\nOptimal Inventory Plan (I):")
for p in products:
    for t in months:
        print(f"I[{p},{t}] = {pulp.value(I[p][t])}", end=", ")
    print()

print("\nTotal Minimized Cost = ", pulp.value(model.objective))

# Also print out total inventory holding cost to confirm it doesn't exceed £500
total_inventory_cost = sum(k[p]*pulp.value(I[p][t]) for p in products for t in months)
print("Total Inventory Holding Cost =", total_inventory_cost)


Status: Optimal

Optimal Production Plan (x):
x[A,1] = 100.0, x[A,2] = 110.0, x[A,3] = 120.0, x[A,4] = 130.0, 
x[B,1] = 80.0, x[B,2] = 90.0, x[B,3] = 100.0, x[B,4] = 110.0, 
x[C,1] = 162.0, x[C,2] = 201.0, x[C,3] = 126.0, x[C,4] = 51.0, 

Optimal Inventory Plan (I):
I[A,1] = 0.0, I[A,2] = 0.0, I[A,3] = 0.0, I[A,4] = 0.0, 
I[B,1] = 0.0, I[B,2] = 0.0, I[B,3] = 0.0, I[B,4] = 0.0, 
I[C,1] = 42.0, I[C,2] = 113.0, I[C,3] = 99.0, I[C,4] = 0.0, 

Total Minimized Cost =  26127.0
Total Inventory Holding Cost = 127.0
